In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from gensim.models import Word2Vec
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import spacy
nlp = spacy.load('en_core_web_md')
# nlp.add_pipe(nlp.create_pipe('sentencizer'))
from utils import *

In [10]:
data = getData()

/home/pranav_pro/SchoolSystem/models/essay/utils.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data['illegible']=data.essay.str.contains('(\?\?\?|illegible|not legible)')


In [11]:
data.describe()

,essay_id,essay_set,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
count,12701.000000,12701.000000,12701.000000,12701.000000,128.000000,12701.000000,1799.000000,1799.000000,1799.000000,2055.000000,...,2055.000000,2055.000000,722.000000,722.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,10146.245020,4.129832,4.057791,4.068341,37.828125,6.635226,3.334074,3.330739,3.334074,2.504136,...,2.708029,2.773723,3.778393,3.590028,3.945312,3.890625,4.078125,3.992188,3.843750,3.617188
std,6266.038381,2.123600,4.205748,4.259040,5.240829,8.945280,0.729263,0.726967,0.729263,1.229647,...,1.168146,1.061814,0.689271,0.693389,0.643668,0.630390,0.622535,0.509687,0.538845,0.603417
min,1.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000
25%,4372.000000,2.000000,2.000000,2.000000,36.000000,2.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,2.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000
50%,9941.000000,4.000000,3.000000,3.000000,40.000000,3.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,15513.000000,6.000000,4.000000,4.000000,40.000000,8.000000,4.000000,4.000000,4.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,21633.000000,8.000000,30.000000,30.000000,50.000000,60.000000,4.000000,4.000000,4.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,5.000000,5.000000


In [12]:
main = data[["essay_id", "essay_set", "essay"]]

In [13]:
w2vModel = Word2Vec.load('Essayw2v.model')
vocab = set(w2vModel.wv.vocab)
dic = w2vModel.wv
w2v = dict(zip(w2vModel.wv.index2word, w2vModel.wv.syn0))

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [14]:
def convert2tokens(essay):
    essay = basicClean(essay)
    essay = replaceNER(essay)
    essay = nlp(essay)
#     essay = [token.text for token in essay if ((not token.is_punct) and (not token.is_stop))]
#     t_essay = [token.text for token in essay if (not token.is_punct)]
    return essay

In [15]:
def getTokens(essay):
    essay = [token.text for token in essay if (not token.is_punct)]
    return essay

In [16]:
tqdm.pandas()
main['spacy'] = main['essay'].progress_apply(lambda x: convert2tokens(x))
main['tokenize'] = main['spacy'].progress_apply(lambda x: getTokens(x))
# main[['spacy_essay','tokenize']] = main['essay'].progress_apply(lambda x: convert2tokens(x))


  3%|▎         | 335/12701 [00:20<13:43, 15.02it/s]


  6%|▌         | 702/12701 [00:39<10:09, 19.68it/s]


  8%|▊         | 1029/12701 [00:58<11:51, 16.41it/s]


 11%|█         | 1363/12701 [01:17<10:15, 18.41it/s]


 14%|█▎        | 1732/12701 [01:38<09:14, 19.77it/s]


...

 93%|█████████▎| 11804/12701 [06:29<00:25, 34.78it/s]


 96%|█████████▌| 12191/12701 [06:52<00:44, 11.47it/s]


 98%|█████████▊| 12480/12701 [07:15<00:17, 12.47it/s]


100%|██████████| 12701/12701 [07:33<00:00, 10.51it/s]/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

 68%|██████▊   | 8611/12701 [00:02<00:00, 4386.60it/s]


100%|██████████| 12701/12701 [00:04<00:00, 3019.53it/s]/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
tokenizer = Tokenizer(nb_words=len(vocab))
tokenizer.fit_on_texts(main['tokenize'])
main['sequence'] = tokenizer.texts_to_sequences(main['tokenize'])
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 42557 unique tokens.


/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# MaxSequence = max([len(seq) for seq in data['sequence']])
MaxSequence = 1200
tqdm.pandas()
main['pad_seq']=list(pad_sequences(main['sequence'], maxlen=MaxSequence))
# data['padSequence'] = data['sequence'].progress_apply(lambda seq: pad_sequences([seq],maxlen=MaxSequence))

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
main.columns

Index(['essay_id', 'essay_set', 'essay', 'spacy', 'tokenize', 'sequence',
       'pad_seq'],
      dtype='object')

In [20]:
avg_len=[350,350,150,150,150,150,250,650]
main['len'] = main.apply(lambda x: len(x['tokenize'])/avg_len[x['essay_set']-1], axis=1)

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
main.head()

,essay_id,essay_set,essay,spacy,tokenize,sequence,pad_seq,len,sent_count,avg_sent_len,punc_count,oov_count,noun_count,verb_count,adv_count,adj_count,det_count,max_depth,avg_depth
0,1,1,"Dear local newspaper, I think effects computer...","(dear, local, newspaper, ,, i, think, effects,...","[dear, local, newspaper, i, think, effects, co...","[302, 485, 427, 7, 67, 524, 42, 20, 18, 21, 19...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000,22,15.909091,41,8,96,71,26,18,32,12,3.987212
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...","(dear, @caps, @caps, i, believe, that, using, ...","[dear, @caps, @caps, i, believe, that, using, ...","[302, 11, 11, 7, 176, 6, 312, 42, 60, 634, 105...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.208571,19,22.263158,29,15,119,100,31,21,45,13,4.482301
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...","(dear, ,, @caps, @caps, @caps, more, and, more...","[dear, @caps, @caps, @caps, more, and, more, p...","[302, 11, 11, 11, 76, 3, 76, 21, 122, 42, 35, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.808571,20,14.150000,22,7,84,59,18,19,30,10,3.531148
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...","(dear, local, newspaper, ,, @caps, i, have, fo...","[dear, local, newspaper, @caps, i, have, found...","[302, 485, 427, 11, 7, 20, 250, 6, 73, 1267, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.511429,44,12.022727,44,49,161,113,38,38,53,10,3.268761
4,5,1,"Dear @LOCATION1, I know having computers has a...","(dear, @location, i, know, having, computers, ...","[dear, @location, i, know, having, computers, ...","[302, 127, 7, 90, 235, 42, 95, 4, 346, 288, 18...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.351429,30,15.766667,45,13,119,114,46,29,61,13,3.559846


In [31]:
def depth(doc, token):
        ans=1
        while token.head!=token:
            token=token.head
            ans+=1
        return ans

In [34]:
weight=[350,350,150,150,150,150,250,650]
main['sent_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len(list(x['spacy'].sents)), axis=1)
main['avg_sent_len'] = main.apply(lambda x: len(x['tokenize'])/x['sent_count'], axis=1)
main['punc_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([token.text for token in x['spacy'] if token.is_punct]), axis=1)
main['oov_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([token.text for token in x['spacy'] if token.is_oov]), axis=1)
main['noun_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([chunk.text for chunk in x['spacy'].noun_chunks]), axis=1)
main['verb_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([token.text for token in x['spacy'] if token.pos_=="VERB"]), axis=1)
main['adv_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([token.text for token in x['spacy'] if token.pos_=="ADV"]), axis=1)
main['adj_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([token.text for token in x['spacy'] if token.pos_=="ADJ"]), axis=1)
main['det_count']=main.apply(lambda x: (10/weight[x['essay_set']-1])*len([token.text for token in x['spacy'] if token.pos_=="DET"]), axis=1)
main['max_depth']=main['spacy'].apply(lambda x: max([depth(x,token) for token in x]))
main['avg_depth']=main['spacy'].apply(lambda x: sum([depth(x,token) for token in x])/len([depth(x,token) for token in x]))
main.describe()

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,essay_id,essay_set,len,sent_count,avg_sent_len,punc_count,oov_count,noun_count,verb_count,adv_count,adj_count,det_count,max_depth,avg_depth
count,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000,12701.000000
mean,10146.245020,4.129832,0.876472,0.615730,372.675523,0.850635,0.270200,2.382564,1.895118,0.671462,0.549330,1.192723,9.261318,3.626002
std,6266.038381,2.123600,0.412418,0.346383,218.887948,0.553527,0.302476,1.125950,0.934079,0.393560,0.342834,0.622380,2.322158,0.597843
min,1.000000,1.000000,0.007692,0.015385,30.000000,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,0.000000,2.000000,1.500000
25%,4372.000000,2.000000,0.560000,0.371429,217.500000,0.400000,0.066667,1.533333,1.200000,0.400000,0.280000,0.733333,8.000000,3.242424
50%,9941.000000,4.000000,0.865714,0.571429,295.454545,0.760000,0.200000,2.333333,1.828571,0.628571,0.514286,1.133333,9.000000,3.560799
75%,15513.000000,6.000000,1.157143,0.800000,487.500000,1.200000,0.371429,3.133333,2.485714,0.933333,0.742857,1.569231,11.000000,3.922222
max,21633.000000,8.000000,3.080000,3.800000,2304.791667,4.200000,4.600000,8.571429,8.542857,3.342857,2.400000,5.400000,28.000000,9.864865


In [35]:
temp = main.copy()
del temp['spacy'], temp['essay']
temp.to_pickle("fullfeatures.pickle")
temp.to_csv("./fullfeatures.csv")
# pickle_out = open("fullfeatures.pickle","wb")
# pickle.dump(temp, pickle_out)